In [1]:
import torch
# from torch.utils.data import Dataset, DataLoader
# from utils.PreProc_Data.DataProc_TM2 import StackedSequenceDataset
# import numpy as np
# import MZA_Experiment
# from MZA_Experiment import MZA_Experiment

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps") 
else:
    device = torch.device("cpu")

Grad testing

In [10]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

In [11]:
Q = 3*a**3 - b**2


In [12]:
external_grad = torch.tensor([1., 1.])

Q.backward(gradient=external_grad)

In [13]:
print(a.grad)
print(b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])


DATA Testing